# Lecture 6: Template attacks

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import numpy as np
import plotly.graph_objects as pgo
import scipy
import lascar

from securec.capture import capture

## 2. Detecting data between noise

In [ ]:
data = capture(
    platform="cwlitexmega",
    number_of_traces=1000, 
    number_of_samples=100,
    code="volatile uint8_t result = input[0];",
    inputfunction=lambda _: [0],
)

In [ ]:
fig = pgo.Figure()
hist = np.histogram(data["trace"][:, 16], bins=10, density=True)
fig.add_trace(pgo.Bar(y=hist[0], x=hist[1]))
fig.show()

In [ ]:
data = capture(
    platform="cwlitexmega",
    number_of_traces=300,
    number_of_samples=100,
    code="volatile uint8_t result = input[0];",
    inputfunction=lambda i: [(0x00, 0xF0, 0xFF)[i // 100]],
)

In [ ]:
fig = pgo.Figure()
for i in range(3):
    d = data["trace"][100 * i : 100 * (i + 1), 16]
    hist = np.histogram(d, bins=10, density=True)
    mu, std = scipy.stats.norm.fit(d)
    fig.add_trace(pgo.Bar(y=hist[0], x=hist[1]))

for i in range(3):
    d = data["trace"][100 * i : 100 * (i + 1), 16]
    mu, std = scipy.stats.norm.fit(d)
    xs = np.linspace(mu - 2 * std, mu + 2 * std)
    fig.add_trace(pgo.Scatter(x=xs, y=scipy.stats.norm.pdf(xs, mu, std)))
fig.show()


## 3. Develop an attack for AES SBox Lookup

In [ ]:
data = capture(
    platform="cwlitexmega",
    number_of_traces=5000, 
    number_of_samples=1000,
    fromfile=os.path.abspath("sbox_lookup_input_key.c"),
    inputfunction=lambda _: [random.randint(0, 255) for _ in range(32)],
)

In [ ]:
def group_traces_by_hw(data, selection_function=lambda trace: lascar.hamming(lascar.tools.aes.sbox[trace["input"][0] ^ trace["input"][16]])):
    groups = {i: [] for i in range(9)}
    for i, trace in enumerate(data):
        groups[selection_function(trace)].append(i)
    return {hw: np.take(data, indices) for hw, indices in groups.items()}

def multivariate_normal(data):
    means = np.mean(data, axis=1)
    covs = np.cov(data)
    return scipy.stats.multivariate_normal(means, covs)

In [ ]:
def generate_template(data, pois=[113, 145, 114]):
    data_grouped = group_traces_by_hw(data)
    return {
        i: multivariate_normal(np.take(data_grouped[i]["trace"], pois, axis=1).T)
        for i in range(9)
    }

In [ ]:
template = generate_template(data)

In [ ]:
data_attack = capture(
    platform="cwlitexmega",
    number_of_traces=50, 
    number_of_samples=1000,
    fromfile=os.path.abspath("sbox_lookup_input_key.c"),
    inputfunction=lambda _: [random.randint(0, 255) for _ in range(16)] + 16 * [99],
)

In [ ]:
def template_attack_sbox_output(data_attack, template, pois=[113, 145, 114]):
    probabilities = np.zeros(256)
    for i, trace in enumerate(data_attack):
        for guess in range(256):
            intermediate = lascar.hamming(
                lascar.tools.aes.sbox[trace["input"][0] ^ guess]
            )
            trace_at_pois = np.take(trace["trace"], pois)
            probabilities[guess] += template[intermediate].logpdf(trace_at_pois)
        print(i, np.argsort(probabilities)[-3:])


template_attack_sbox_output(data_attack, template)


## 4. Find Points of Interest

In [ ]:
import itertools

def grouped_diffs(data_grouped):
    means = {hw: np.nan_to_num(np.nanmean(data_grouped[hw]["trace"], axis=0)) for hw in range(9)}
    return sum(
        (meani - meanj) ** 2
        for meani, meanj
        in itertools.product(means.values(), repeat=2)
    )

diffs = grouped_diffs(group_traces_by_hw(data))
pois = np.argsort(diffs)[-5:]

fig = pgo.Figure()
fig.add_trace(pgo.Scatter(y=diffs))
fig.show()

pois


In [ ]:
template_attack_sbox_output(data_attack, generate_template(data, pois=pois), pois=pois)

## 5. Template attack using `sklearn.discriminant_analysis` and `lascar`

In [ ]:
import sklearn
import sklearn.discriminant_analysis

In [ ]:
def generate_template_lascar(
    data,
    pois,
    selection_function=lambda value: lascar.hamming(
        lascar.tools.aes.sbox[value["input"][0] ^ value["input"][16]]
    ),
):
    classifier_qda = sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis(
        store_covariance=True
    )

    classifier_profile_engine_qda = lascar.ProfileEngine(
        classifier=classifier_qda,
        partition_function=selection_function,
        partition_range=range(9),
    )

    trace = lascar.TraceBatchContainer(data["trace"], data)
    trace.leakage_section = pois
    session = lascar.Session(
        trace,
        engine=classifier_profile_engine_qda,
    ).run()
    return classifier_qda

In [ ]:
template_lascar = generate_template_lascar(data, pois=pois)

In [ ]:
def template_attack_sbox_output_lascar(
    data_attack,
    template,
    pois,
    selection_function=lambda value, guess: lascar.hamming(
        lascar.tools.aes.sbox[value["input"][0] ^ guess]
    ),
):
    classifier_match_engine_qda = lascar.MatchEngine(
        classifier=template,
        selection_function=selection_function,
        guess_range=range(256),
    )

    trace = lascar.TraceBatchContainer(data_attack["trace"], data_attack)
    trace.leakage_section = pois
    session = lascar.Session(
        trace,
        engine=classifier_match_engine_qda,
        output_method=lascar.MultipleOutputMethod(
            lascar.ScoreProgressionOutputMethod(classifier_match_engine_qda),
            lascar.ConsoleOutputMethod(classifier_match_engine_qda),
        ),
        output_steps=range(0, 50, 1),
    ).run(thread_on_update=False)


In [ ]:
template_attack_sbox_output_lascar(data_attack, template_lascar, pois)